# Functions and Imports - no pyiron dependence

In [1]:
from h5io_browser import Pointer
from h5io_browser.base import _open_hdf, _read_hdf
import h5py
import posixpath
import os

In [2]:
def list_groups_and_nodes(hdf, h5_path):
    """
    Get the list of groups and list of nodes from an open HDF5 file

    Args:
        hdf (h5py.File): file handle of an open HDF5 file
        h5_path (str): path inside the HDF5 file

    Returns:
        list, list: list of groups and list of nodes
    """
    groups = set()
    nodes = set()
    try:
        h = hdf[h5_path]
        for k in h.keys():
            if isinstance(h[k], h5py.Group):
                group_attrs_dict = h[k].attrs
                if 'TITLE' in group_attrs_dict.keys() and group_attrs_dict['TITLE'] == 'multiarray':
                    nodes.add(k)
                else:
                    groups.add(k)
            else:
                nodes.add(k)
    except KeyError:
        pass
    return list(groups), list(nodes)

In [3]:
def get_h5_path(h5_path, name):
    """
    Combine the current h5_path with the relative path

    Args:
        h5_path (str): absolute path of the node in the hdf5 file
        name (str): relative path to be added to the absolute path

    Returns:
        str: combined path
    """
    return posixpath.join(h5_path, name)

In [4]:
def read_dict_from_hdf(
    file_name, h5_path, recursive=False, group_paths=[], slash="ignore"
):
    """
    Read data from HDF5 file into a dictionary - by default only the nodes are converted to dictionaries, additional
    sub groups can be specified using the group_paths parameter.
    Args:
       file_name (str): Name of the file on disk
       h5_path (str): Path to a group in the HDF5 file from where the data is read
       recursive (bool): Load all subgroups recursively
       group_paths (list): list of additional groups to be included in the dictionary, for example:
                           ["input", "output", "output/generic"]
                           These groups are defined relative to the h5_path.
       slash (str): 'ignore' | 'replace' Whether to replace the string {FWDSLASH} with the value /. This does
                    not apply to the top level name (title). If 'ignore', nothing will be replaced.
    Returns:
       dict:     The loaded data. Can be of any type supported by ``write_hdf5``.
    """

    def get_dict_from_nodes(store, h5_path, slash="ignore"):
        """
        Load all nodes from an HDF5 path into a dictionary
        Args:
            store (str): Name of the file on disk, or file-like object.  Note: for files created with the 'core'
                         driver, HDF5 still requires this be non-empty.:
            h5_path (str): Path to a group in the HDF5 file from where the data is read
            slash (str): 'ignore' | 'replace' Whether to replace the string {FWDSLASH} with the value /. This does
                         not apply to the top level name (title). If 'ignore', nothing will be replaced.
        Returns:
            dict:        The loaded data. Can be of any type supported by ``write_hdf5``.
        """
        return {
            n: _read_hdf(
                hdf_filehandle=store,
                h5_path=get_h5_path(h5_path=h5_path, name=n),
                slash=slash,
            )
            for n in list_groups_and_nodes(hdf=store, h5_path=h5_path)[1]
        }

    def resolve_nested_dict(group_path, data_dict):
        """
        Turns a dict with a key containing slashes into a nested dict.  {'/a/b/c': 1} -> {'a': {'b': {'c': 1}
        Args:
            group_path (str): path inside the HDF5 file the data_dictionary was loaded from
            data_dict (dict): dictionary with data loaded from the HDF5 file
        Returns:
            dict: hierarchical dictionary
        """
        groups = group_path.split("/")
        nested_dict = data_dict
        for g in groups[::-1]:
            nested_dict = {g: nested_dict}
        return nested_dict

    def get_groups_hdf(hdf, h5_path):
        """
        Get all sub-groups of a given HDF5 path
        Args:
            hdf (str): Name of the file on disk, or file-like object.  Note: for files created with the 'core'
                       driver, HDF5 still requires this be non-empty.:
            h5_path (str): Path to a group in the HDF5 file from where the data is read
        Returns:
            list: list of HDF5 groups
        """
        try:
            h = hdf[h5_path]
            group_lst = []
            for group in [h[k].name for k in h.keys() if isinstance(h[k], h5py.Group)]:
                group_attrs_dict = h[group].attrs
                if 'TITLE' in group_attrs_dict.keys() and group_attrs_dict['TITLE'] != 'multiarray':
                    group_lst += [group] + get_groups_hdf(hdf=hdf, h5_path=group)
                elif 'TITLE' not in group_attrs_dict.keys():
                    group_lst += [group] + get_groups_hdf(hdf=hdf, h5_path=group)
            return group_lst
        except KeyError:
            return []

    def merge_dict(main_dict, add_dict):
        """
        Merge two dictionaries recursively

        Args:
            main_dict (dict): The primary dictionary, the secondary dictionary is merged into
            add_dict (dict): The secondary dictionary which is merged in the primary dictionary

        Returns:
            dict: The merged dictionary with all keys
        """
        for k, v in add_dict.items():
            if k in main_dict.keys() and isinstance(v, dict):
                main_dict[k] = merge_dict(main_dict=main_dict[k], add_dict=v)
            else:
                main_dict[k] = v
        return main_dict

    if recursive and len(group_paths) > 0:
        raise ValueError(
            "Loading subgroups can either be defined by the group paths ",
            group_paths,
            " or by the recursive ",
            recursive,
            " parameter. Specifying both lead to this ValueError.",
        )

    with _open_hdf(file_name, mode="r") as store:
        output_dict = get_dict_from_nodes(store=store, h5_path=h5_path, slash=slash)
        if h5_path == "/" and recursive:
            group_paths = [g[1:] for g in get_groups_hdf(hdf=store, h5_path=h5_path)]
        elif h5_path[0] != "/" and recursive:
            group_paths = [
                g[len("/" + h5_path) + 1 :]
                for g in get_groups_hdf(hdf=store, h5_path="/" + h5_path)
            ]
        elif recursive:
            group_paths = [
                g[len(h5_path) + 1 :]
                for g in get_groups_hdf(hdf=store, h5_path=h5_path)
            ]
        for group_path in group_paths:
            output_dict = merge_dict(
                main_dict=output_dict,
                add_dict=resolve_nested_dict(
                    group_path=group_path,
                    data_dict=get_dict_from_nodes(
                        store=store,
                        h5_path=get_h5_path(h5_path=h5_path, name=group_path),
                        slash=slash,
                    ),
                ),
            )
    return output_dict

# Create VASP job

In [5]:
from pyiron_atomistics import Project

In [6]:
pr = Project("test")

In [7]:
pr.remove_jobs(recursive=True, silently=True)

0it [00:00, ?it/s]

In [8]:
structure = pr.create.structure.ase.bulk("Al", cubic=True)

In [9]:
len(structure)

4

In [10]:
job = pr.create.job.Vasp("vasp")

In [11]:
job.structure =structure

In [12]:
job.calc_minimize()

In [13]:
job.run()

The job vasp was saved and received the ID: 22805519


# Read job dictionary

In [14]:
job_dict = read_dict_from_hdf(
    file_name=job.project_hdf5.file_name,
    h5_path="/",
    recursive=True,
    group_paths=[],
    slash='ignore',
)

# Get VASP schema

In [15]:
file_name = job.project_hdf5.file_name
file_name

'/cmmc/u/janj/notebooks/2024/2024-06-06-read-hdf/test/vasp.h5'

In [16]:
key_dict = {}
def collect_attrs(name, obj):
    if len(obj.attrs) != 0:
        key_dict[name] = obj.attrs["TITLE"]
    
with h5py.File(file_name, 'r') as f:
    f.visititems(collect_attrs)

In [17]:
group_lst, node_lst = [], []
for k, v in key_dict.items():
    if len([g for g in group_lst if g in k]) == 0:
        node_lst.append(k)
        if v == "multiarray":
            group_lst.append(k)
node_lst

['vasp/DICT_VERSION',
 'vasp/HDF_VERSION',
 'vasp/NAME',
 'vasp/OBJECT',
 'vasp/TYPE',
 'vasp/VERSION',
 'vasp/executable',
 'vasp/input/generic/DICT_VERSION',
 'vasp/input/generic/NAME',
 'vasp/input/generic/OBJECT',
 'vasp/input/generic/TYPE',
 'vasp/input/generic/VERSION',
 'vasp/input/generic/data_dict',
 'vasp/input/generic_dict',
 'vasp/input/incar/DICT_VERSION',
 'vasp/input/incar/NAME',
 'vasp/input/incar/OBJECT',
 'vasp/input/incar/TYPE',
 'vasp/input/incar/VERSION',
 'vasp/input/incar/data_dict',
 'vasp/input/interactive',
 'vasp/input/kpoints/DICT_VERSION',
 'vasp/input/kpoints/NAME',
 'vasp/input/kpoints/OBJECT',
 'vasp/input/kpoints/TYPE',
 'vasp/input/kpoints/VERSION',
 'vasp/input/kpoints/data_dict',
 'vasp/input/potcar/DICT_VERSION',
 'vasp/input/potcar/NAME',
 'vasp/input/potcar/OBJECT',
 'vasp/input/potcar/TYPE',
 'vasp/input/potcar/VERSION',
 'vasp/input/potcar/data_dict',
 'vasp/input/potential_dict',
 'vasp/input/structure/TYPE',
 'vasp/input/structure/cell/cell',


# Write VASP job

In [18]:
def get_node_from_job_dict(job_dict, node):
    node_name_lst = node.split("/")
    tmp_dict = job_dict
    for group in node_name_lst:
        tmp_dict = tmp_dict[group]
    return tmp_dict

In [19]:
node_dict = {node: get_node_from_job_dict(job_dict=job_dict, node=node) for node in node_lst}

In [20]:
os.remove(file_name)

In [21]:
with Pointer(file_name=file_name) as hdf_file:
    hdf_file.write_dict(node_dict)

# Reload job

In [22]:
job = pr.load(job.job_name)

In [23]:
job.input.incar

,Parameter,Value,Comment
0,SYSTEM,vasp,jobname
1,PREC,Accurate,
2,ALGO,Fast,
3,LREAL,False,
4,LWAVE,False,
5,LORBIT,0,
6,ISIF,2,
7,IBRION,2,
8,NELM,400,
9,NSW,100,


In [24]:
job.input.kpoints

,Parameter,Value,Comment
0,,Kpoints file generated with pyiron_atomistics,
1,,0,
2,,Monkhorst_Pack,
3,,4 4 4,
4,,0 0 0,


In [25]:
job.input.potcar

,Parameter,Value,Comment
0,xc,GGA,"LDA, GGA"


In [26]:
job.output.energy_tot

array([-14.7459202])

In [27]:
job.output.positions

array([[[0.   , 0.   , 0.   ],
        [0.   , 2.025, 2.025],
        [2.025, 0.   , 2.025],
        [2.025, 2.025, 0.   ]]])